**Алгоритм обработки отчетов по продажам всех популярных маркетплейсов** 

Алгоритм подготавливает файл для формирования дашборда в Tableau и обновляет общую гугл таблицу "План-факт-прогноз" по месяцам.

Все конфиденциальные данные изменены.

Дашборд в Tableau: <https://public.tableau.com/app/profile/alexey.chernov/viz/-_16869947210590/-_2>

Гугл таблица: <https://docs.google.com/spreadsheets/d/1FN58-jfsO6r34gc0CUSPdo1ouJoXXrOFCEWGMnY1I7I/edit#gid=252614967>

In [8]:
# Импортируем нужные библиотеки, запустим подключения и прочитаем датафреймы
import pandas as pd
import datetime
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from pandas.io.json import json_normalize
from df2gspread import df2gspread as d2g
scope = ['https://spreadsheets.google.com/feeds']
credentials = ServiceAccountCredentials.from_json_keyfile_name('totemic-vim-366006-ed79027e67bd.json', scope)
gc = gspread.authorize(credentials)
pd.set_option('display.float_format', '{:.2f}'.format)
ozon = pd.read_excel('ozon.xlsx')
wb = pd.read_excel('wb.xlsx')
sber = pd.read_excel('sber.xls')
sber_pi = pd.read_excel('sber_pi.xls')
ya_fby = pd.read_excel('яндекс fby.xlsx')
ya_dsc = pd.read_excel('яндекс дисконт.xlsx')
ya_fbs_msk_basic = pd.read_excel('яндекс мск основной.xlsx')
ya_fbs_msk_express = pd.read_excel('яндекс мск экспресс.xlsx')
ya_fbs_msk_pi = pd.read_excel('яндекс мск пи.xlsx')
ya_fbs_nsk_basic = pd.read_excel('яндекс нск.xlsx')
ya_fbs_nsk_express = pd.read_excel('яндекс нск экспресс.xlsx')
#ya_basic = pd.read_excel('яндекс основной.xlsx', sheet_name='Транзакции по заказам и товарам', skiprows=7)
#ya_dsc = pd.read_excel('яндекс дисконт.xlsx', sheet_name='Транзакции по заказам и товарам', skiprows=7)
ali = pd.read_excel('али.xlsx')
dm = pd.read_excel('дм.xlsx')
sku = pd.read_excel('sku.xlsx')
past_months = pd.read_excel('past_months.xlsx')

In [9]:
# Для обработки всех датафреймов
use_columns = ["Дата", "Артикул", "Наименование товара", "Количество", "Сумма"]
sku_new = sku[['Артикул', 'Название линейки', 'Штук в упаковке', 'Бренд', 'Номенклатура']]
use_contract = ['Бренд 1', 'Бренд 2', 'Бренд 3', 'Бренд 4', 'Бренд 8', 'Бренд 11']

# Переменная с количеством дней в месяце - меняем!
day_in_month = 31

# Переменная с номером месяца - меняем!
month = 5

In [10]:
# Обработка датафрейма ozon
# Оставим нужные столбцы
ozon_use_columns = [
    'Дата начисления', 
    'Артикул', 
    'Название товара или услуги', 
    'Количество', 
    'За продажу или возврат до вычета комиссий и услуг']
ozon = ozon[ozon_use_columns]

# Переименуем столбцы 
ozon.columns = use_columns

# Удалим нули в столбце "Сумма"
ozon = ozon.query('Сумма != 0')

# Приведем столбец "Артикул" к числовому формату
ozon['Артикул'] = ozon['Артикул'].astype(int)

# Добавим по артикулу дополнительные поля
ozon = pd.merge(ozon, sku_new, how='left', on=('Артикул'))

In [11]:
# Обработка датафрейма wb
# Оставим нужные столбцы
wb_use_columns = [
    'Дата', 
    'Артикул WB', 
    'Название', 
    'Продажи (WB)', 
    'Суммарная выручка (WB)']
wb = wb[wb_use_columns]

# Переименуем столбцы 
wb.columns = use_columns

# Удалим нули в столбце "Сумма"
wb = wb.query('Сумма != 0')

# Приведем столбец "Дата" к временному формату
wb['Дата'] = wb['Дата'].astype('datetime64')

# Добавим по артикулу дополнительные поля
wb = pd.merge(wb, sku_new, how='left', on=('Артикул'))

In [12]:
# Обработка датафрейма sber
# Объединим датафреймы
sber = sber.append(sber_pi)

# Оставим нужные статусы
sber = sber.query('`Статус заказа` == "Доставлен" or `Статус заказа` == "Выполняется"')

# Оставим нужные столбцы
sber_use_columns = [
    'Дата создания заказа', 
    'Артикул товара продавца', 
    'Наименование товара', 
    'Кол-во штук в заказе', 
    'Стоимость товара продавца']
sber = sber[sber_use_columns]

# Переименуем столбцы 
sber.columns = use_columns

# Удалим время в столбце
sber['Дата'] = pd.to_datetime(sber['Дата']).dt.date
sber['Дата'] = pd.to_datetime(sber['Дата'], format='%Y-%m-%d')

# Добавим по артикулу дополнительные поля
sber = pd.merge(sber, sku_new, how='left', on=('Артикул'))

/var/folders/8r/yntrmdj50090fw23b86grqz00000gn/T/ipykernel_1118/3779298260.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sber = sber.append(sber_pi)


In [13]:
# Обработка датафрейма yandex по отчету "Показы и продажи"
# Объединим 7 датафреймов в один
yandex = (
    ya_fby.append(ya_dsc)
    .append(ya_fbs_msk_basic)
    .append(ya_fbs_msk_express)
    .append(ya_fbs_msk_pi)
    .append(ya_fbs_nsk_basic)
    .append(ya_fbs_nsk_express)
)

# Оставим нужные столбцы
ya_use_columns = [
    'День', 
    'Ваш SKU', 
    'Название товара', 
    'Продажи, шт.', 
    'Продажи, руб.'
]
yandex = yandex[ya_use_columns]

# Переименуем столбцы 
yandex.columns = use_columns

# Удалим нули в столбце "Сумма"
yandex = yandex.query('Сумма != 0')

# Приведем столбец "Дата" к временному формату
yandex['Дата'] = pd.to_datetime(yandex['Дата'], format='%d-%m-%Y')

# Добавим по артикулу дополнительные поля
yandex = pd.merge(yandex, sku_new, how='left', on=('Артикул'))

# Обработаем также датафрейм со складом пи
yandex_pi = ya_fbs_msk_pi[ya_use_columns]
yandex_pi.columns = use_columns
yandex_pi = yandex_pi.query('Сумма != 0')
yandex_pi['Дата'] = pd.to_datetime(yandex_pi['Дата'], format='%d-%m-%Y')
yandex_pi = pd.merge(yandex_pi, sku_new, how='left', on=('Артикул'))

/var/folders/8r/yntrmdj50090fw23b86grqz00000gn/T/ipykernel_1118/2429730305.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ya_fby.append(ya_dsc)
/var/folders/8r/yntrmdj50090fw23b86grqz00000gn/T/ipykernel_1118/2429730305.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ya_fby.append(ya_dsc)
/var/folders/8r/yntrmdj50090fw23b86grqz00000gn/T/ipykernel_1118/2429730305.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ya_fby.append(ya_dsc)
/var/folders/8r/yntrmdj50090fw23b86grqz00000gn/T/ipykernel_1118/2429730305.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ya_fby.append(ya_dsc)
/var/folders/8r/yntrmdj50090fw23b86grqz00000gn/T

In [8]:
# Обработка датафрейма yandex по отчету по заказам
#yandex = ya_basic.append(ya_dsc)

# Оставим только статус "Доставлен"
#yandex = yandex.query('`Статус товара` == "Доставлен"')

# Приведем столбец "Статус изменен" в даты и оставим целевой месяц
#yandex['Статус изменен'] = pd.to_datetime(yandex['Статус изменен']).dt.date
#yandex['Статус изменен'] = pd.to_datetime(yandex['Статус изменен'], format='%Y-%m-%d')
#yandex['Месяц'] = yandex['Статус изменен'].dt.month
#yandex = yandex.query('Месяц == @month')

# Сделаем столбец с суммой продажи
#yandex['Сумма'] = yandex['Количество'] * yandex['Ваша цена (за шт.)']

# Сделаем датафрейм отдельно со складом - Пи
#yandex_pi = yandex.query('`Названия магазинов` == "МСК FBS Пи"')

# Оставим нужные столбцы
#ya_use_columns = [
#    'Статус изменен', 
#    'Ваш SKU', 
#    'Название товара', 
#    'Количество', 
#    'Сумма']
#yandex = yandex[ya_use_columns]
#yandex_pi = yandex_pi[ya_use_columns]

# Переименуем столбцы 
#yandex.columns = use_columns
#yandex_pi.columns = use_columns

# Добавим по артикулу дополнительные поля
#yandex = pd.merge(yandex, sku_new, how='left', on=('Артикул'))
#yandex_pi = pd.merge(yandex_pi, sku_new, how='left', on=('Артикул'))

# Уберем неполную дату
#ozon_date = ozon["Дата"].max()
#yandex = yandex.query('Дата <= @ozon_date')
#yandex_pi = yandex_pi.query('Дата <= @ozon_date')

In [14]:
# Обработка датафрейма ali
# Удалим отмененные заказы
ali = ali.query('Отменен != "V"')

# Добавим столбец с суммой
ali['Сумма'] = ali['Количество'] * ali['Цена товара']

# Оставим нужные столбцы
ali_use_columns = [
    'Дата заказа', 
    'Артикул', 
    'Номенклатура, Характеристика', 
    'Количество', 
    'Сумма']
ali = ali[ali_use_columns]

# Переименуем столбцы 
ali.columns = use_columns

# Приведем столбец "Дата" к временному формату
ali['Дата'] = pd.to_datetime(ali['Дата'], format='%d.%m.%Y')

# Добавим по артикулу дополнительные поля
ali = pd.merge(ali, sku_new, how='left', on=('Артикул'))

In [15]:
# Обработка датафрейма dm
# Удалим нули
dm = dm.query('`Сумма заказов в рублях за выбранный период в фактических ценах продажи` != 0')

# Оставим нужные столбцы
dm_use_columns = [
    'Дата', 
    'Артикул по учетной системе поставщика', 
    'Полное наименование товара', 
    'Кол-во заказов в штуках за выбранный период', 
    'Сумма заказов в рублях за выбранный период в фактических ценах продажи']
dm = dm[dm_use_columns]

# Переименуем столбцы 
dm.columns = use_columns

# Добавим по артикулу дополнительные поля
dm = pd.merge(dm, sku_new, how='left', on=('Артикул'))

In [16]:
# Выведем на экран уникальные артикулы, которые не нашлись в sku
nan_ozon = ozon[ozon['Название линейки'].isna()]
nan_wb = wb[wb['Название линейки'].isna()]
nan_sber = sber[sber['Название линейки'].isna()]
nan_yandex = yandex[yandex['Название линейки'].isna()]
nan_ali = ali[ali['Название линейки'].isna()]
nan_dm = dm[dm['Название линейки'].isna()]
nan = nan_ozon.append(nan_wb).append(nan_sber).append(nan_yandex).append(nan_ali).append(nan_dm)
nan = nan[['Артикул', 'Наименование товара']].drop_duplicates(['Артикул']).sort_values(by='Артикул')
nan

/var/folders/8r/yntrmdj50090fw23b86grqz00000gn/T/ipykernel_1118/1486591859.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  nan = nan_ozon.append(nan_wb).append(nan_sber).append(nan_yandex).append(nan_ali).append(nan_dm)
/var/folders/8r/yntrmdj50090fw23b86grqz00000gn/T/ipykernel_1118/1486591859.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  nan = nan_ozon.append(nan_wb).append(nan_sber).append(nan_yandex).append(nan_ali).append(nan_dm)
/var/folders/8r/yntrmdj50090fw23b86grqz00000gn/T/ipykernel_1118/1486591859.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  nan = nan_ozon.append(nan_wb).append(nan_sber).append(nan_yandex).append(nan_ali).append(nan_dm)
/var/folders/8r/yntrmdj50090fw23b86grqz00000gn/T/ipykernel_11

,Артикул,Наименование товара


In [17]:
# Выгрузим в эксель sku новые артикулы, которые не нашлись
sku = sku.append(nan).drop(columns='Себестоимость', axis=1).rename(columns={"Себестоимость новая": "Себестоимость"})
sku.to_excel('sku.xlsx', index=False)

/var/folders/8r/yntrmdj50090fw23b86grqz00000gn/T/ipykernel_1118/1316598748.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sku = sku.append(nan).drop(columns='Себестоимость', axis=1).rename(columns={"Себестоимость новая": "Себестоимость"})


In [18]:
# Создадим столбец с названием маркетплэйса
ozon['Маркетплэйс'] = 'Озон'
wb['Маркетплэйс'] = 'WB'
sber['Маркетплэйс'] = 'Сбер'
yandex['Маркетплэйс'] = 'Яндекс'
ali['Маркетплэйс'] = 'Али'
dm['Маркетплэйс'] = 'ДМ'
yandex_pi['Маркетплэйс'] = 'Яндекс - Пи'

# Создадим сводные таблицы по бренду в каждом маркетплэйсе по сумме и добавим прогноз
# ozon
ozon_total = (
    ozon.groupby(['Маркетплэйс', 'Бренд'])
    .agg({'Сумма': 'sum'})
    .query('`Бренд` in @use_contract')
)
ozon_total['Прогноз'] = ozon_total['Сумма'] / ozon['Дата'].max().day * day_in_month

# wb
wb_total = (
    wb.groupby(['Маркетплэйс', 'Бренд'])
    .agg({'Сумма': 'sum'})
    .query('`Бренд` in @use_contract')
)
wb_total['Прогноз'] = wb_total['Сумма'] / wb['Дата'].max().day * day_in_month

# sber
sber_total = (
    sber.groupby(['Маркетплэйс', 'Бренд'])
    .agg({'Сумма': 'sum'})
    .query('`Бренд` in @use_contract')
)
sber_total['Прогноз'] = sber_total['Сумма'] / sber['Дата'].max().day * day_in_month

# yandex
yandex_total = (
    yandex.groupby(['Маркетплэйс', 'Бренд'])
    .agg({'Сумма': 'sum'})
    .query('`Бренд` in @use_contract')
)
yandex_total['Прогноз'] = yandex_total['Сумма'] / yandex['Дата'].max().day * day_in_month

yandex_pi_total = (
    yandex_pi.groupby(['Маркетплэйс', 'Бренд'])
    .agg({'Сумма': 'sum'})
    .query('`Бренд` in @use_contract')
)
yandex_pi_total['Прогноз'] = yandex_pi_total['Сумма'] / yandex_pi['Дата'].max().day * day_in_month

# ali
ali_total = (
    ali.groupby(['Маркетплэйс', 'Бренд'])
    .agg({'Сумма': 'sum'})
    .query('`Бренд` in @use_contract')
)
ali_total['Прогноз'] = ali_total['Сумма'] / ali['Дата'].max().day * day_in_month

# dm
dm_total = (
    dm.groupby(['Маркетплэйс', 'Бренд'])
    .agg({'Сумма': 'sum'})
    .query('`Бренд` in @use_contract')
)
dm_total['Прогноз'] = dm_total['Сумма'] / dm['Дата'].max().day * day_in_month

# Создадим общую сводную
sales = ozon_total.append(wb_total).append(sber_total).append(yandex_total).append(ali_total).append(dm_total).append(yandex_pi_total).reset_index()
sales['Сцепка'] = sales['Маркетплэйс'] + sales['Бренд']
sales['Сумма'] = sales['Сумма'].astype(int)
sales['Прогноз'] = sales['Прогноз'].astype(int)
sales = sales[['Сцепка', 'Маркетплэйс', 'Бренд', 'Сумма', 'Прогноз']].reset_index(drop=True)

/var/folders/8r/yntrmdj50090fw23b86grqz00000gn/T/ipykernel_1118/2294618282.py:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sales = ozon_total.append(wb_total).append(sber_total).append(yandex_total).append(ali_total).append(dm_total).append(yandex_pi_total).reset_index()
/var/folders/8r/yntrmdj50090fw23b86grqz00000gn/T/ipykernel_1118/2294618282.py:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sales = ozon_total.append(wb_total).append(sber_total).append(yandex_total).append(ali_total).append(dm_total).append(yandex_pi_total).reset_index()
/var/folders/8r/yntrmdj50090fw23b86grqz00000gn/T/ipykernel_1118/2294618282.py:67: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sales = ozon_total.append(wb_total).append(sber_tot

In [19]:
# Выгружаем таблицу в гугл "Маркетплейсы: Продажи"
spreadsheet_key = '1FN58-jfsO6r34gc0CUSPdo1ouJoXXrOFCEWGMnY1I7I'
wks_name = 'Юпитер - импорт'
d2g.upload(sales, spreadsheet_key, wks_name, credentials=credentials, row_names=True)

<Worksheet 'Юпитер - импорт' id:1686658842>

In [20]:
# Создадим файл с последней датой каждого маркетплэйса и выгрузим в гугл "МП: Продажи 2023"
last_day = (
    ozon.append(wb)
    .append(sber)
    .append(yandex)
    .append(yandex_pi)
    .append(ali)
    .append(dm)
    .groupby('Маркетплэйс')
    .agg({'Дата': 'max'})
    .reset_index()
)
wks_name = 'Юпитер - даты'
d2g.upload(last_day, spreadsheet_key, wks_name, credentials=credentials, row_names=True)

/var/folders/8r/yntrmdj50090fw23b86grqz00000gn/T/ipykernel_1118/397736374.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ozon.append(wb)
/var/folders/8r/yntrmdj50090fw23b86grqz00000gn/T/ipykernel_1118/397736374.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ozon.append(wb)
/var/folders/8r/yntrmdj50090fw23b86grqz00000gn/T/ipykernel_1118/397736374.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ozon.append(wb)
/var/folders/8r/yntrmdj50090fw23b86grqz00000gn/T/ipykernel_1118/397736374.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ozon.append(wb)
/var/folders/8r/yntrmdj50090fw23b86grqz00000gn/T/ipykernel_1118/397736374.py

<Worksheet 'Юпитер - даты' id:1021567636>

In [22]:
# Подготовим и выгрузим файл в эксель для дашборда в Табло
tableau = (
    ozon.append(wb)
    .append(sber)
    .append(yandex)
    .append(ali)
    .append(dm)
    .groupby(['Маркетплэйс', 'Бренд', 'Название линейки', 'Номенклатура', 'Дата'])
    .agg({'Сумма': 'sum', 'Количество': 'sum'})
    .query('`Бренд` in @use_contract')
    .reset_index()
)
tableau['Сцепка'] = tableau['Маркетплэйс'] + tableau['Бренд']
def contract_group(row):
    con = row['Бренд']
    if con == 'Бренд 2' or con == 'Бренд 11':
        return 'Бренд 2+11'
    else: 
        return 'Контракты'
sales_tableau = sales[['Сцепка', 'Прогноз']]
tableau = pd.merge(tableau, sales_tableau, how='left', on=('Сцепка'))
tableau['Группа по контрактам'] = tableau.apply(contract_group, axis=1)
prognos_group = (
    tableau[['Прогноз', 'Группа по контрактам']].drop_duplicates()
    .groupby('Группа по контрактам')
    .agg({'Прогноз':'sum'})
    .rename(columns={"Прогноз": "Прогноз по группе"})
)
tableau = pd.merge(tableau, prognos_group, how='left', on=('Группа по контрактам'))
book = gc.open_by_key(spreadsheet_key)
worksheet = book.worksheet("Export")
table = worksheet.get_all_values()
plan = pd.DataFrame(table[1:], columns=table[0])
plan = plan[['Сцепка', 'Маркетплэйс', 'Бренд', 'План']]
plan = plan.apply(pd.to_numeric, errors='ignore')
plan['Группа по контрактам'] = plan.apply(contract_group, axis=1)
plan_group = plan.groupby('Группа по контрактам').agg({'План':'sum'}).rename(columns={"План": "План по группе"})
plan = pd.merge(plan, plan_group, how='left', on=('Группа по контрактам'))
plan = plan[['Сцепка', 'План', 'План по группе']]
tableau = pd.merge(tableau, plan, how='left', on=('Сцепка'))
tableau = tableau.drop(columns = ['Сцепка'],axis = 1).append(past_months)
tableau.to_excel('tableau.xlsx', index=False)

/var/folders/8r/yntrmdj50090fw23b86grqz00000gn/T/ipykernel_1118/1620154173.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ozon.append(wb)
/var/folders/8r/yntrmdj50090fw23b86grqz00000gn/T/ipykernel_1118/1620154173.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ozon.append(wb)
/var/folders/8r/yntrmdj50090fw23b86grqz00000gn/T/ipykernel_1118/1620154173.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ozon.append(wb)
/var/folders/8r/yntrmdj50090fw23b86grqz00000gn/T/ipykernel_1118/1620154173.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ozon.append(wb)
/var/folders/8r/yntrmdj50090fw23b86grqz00000gn/T/ipykernel_1118/16201541

In [17]:
# По завершении месяца выгрузить эксель past_months
#tableau.to_excel('past_months.xlsx', index=False)